# Latching Power Circuit for Raspberry Pi - Design Justification

I wanted to add a way to control the power in one of my projects. It's an old radio that has had it's internals removed and replaced with a Raspberry Pi and an amp. I wanted to use the knobs in some meaningful way, so I thought on starting with power and volume.

## Volume Knob

Let's start with volume because it's pretty straightforward. A rotary encoder with a push button. The only decision to make here is what's the time constant of the RC filter used to debounce the switches.

## Latching Power Circuit

The Raspberry Pi is unable to control the power provided to the board. You can "shut down" the Raspberry Pi, but that won't stop the current draw (although it will reduce it). Also, when shutting down the Pi, you have to remove the power cable and plug it in back again to power it up again. Urgh!

The circuit I used as a reference can be found [here](http://www.mosaic-industries.com/embedded-systems/microcontroller-projects/electronic-circuits/push-button-switch-turn-on/latching-toggle-power-switch).

[TODO: Add schematic]

In the initial state, with the Auto-On feature turned off the NMOS (Q1A) is off (the only element driving that side of the circuit is the now-unpowered regulator's output, the gate is floating). As it is, the NMOS has no influence in V<sub>A</sub>. The gate of the PMOS is connected to the C1 Capacitor. The diode D1 is basically floating, and C1 has both pins connected to V<sub>IN</sub> via two resistors, so it remains discharged. One of the initial conditions of the analysis is that the Auto-on feature is set to power off, so the C2 cap is connected to V<sub>IN</sub>, and Vgs will be 0V and the PMOS won't turn on on power up.

If the switch is momentarily flicked, it connects the PWR_SWITCH net to ground. One of the sides of C1 is pulled to ground and starts to charge towards V<sub>IN</sub> via R2 (V<sub>C1</sub> = V<sub>A</sub> - V<sub>PWR_SWITCH</sub>). Because it starts discharged, its drop is 0V, so the gate of the PMOS is pulled to ground too and the PMOS starts to conduct. With power on the output (V<sub>OUT</sub>) net and no voltage drop in C4 (is discharged too) the NMOS gate is pulled high and starts to conduct, holding the gate of the PMOS low and conducting. The NMOS is the latch. C1 stops charging as both sides are pulled to ground now. Releasing the switch will again leave the V<sub>PWR_SWITCH</sub> floating, and C1 will charge to V<sub>IN</sub> via R1. This all is instant, notice the timescale in the graph. C4 briefly charges, but its voltage is effectively 0 (low 100s of mV). Even so, when the switch is released, it loses its path to ground and will leak the voltage until it settles in V<sub>C4</sub> = 0V.

[PoweringOn graph]

Another press of the power button when the output is powered will get the PWR_SWITCH net again pulled to ground. This time, the diode D1 will get the R7-R8-R9 net to 0V + V<sub>f</sub> (~0.4V). This would be enough to shut the NMOS off, but the gate of the NMOS will be held to 3V3 because C4 is "effectively" discharged and tying it to 3V3, but this will give C4 a path to charge, so it will only hold the the voltage for a certain amount of time. This is the "protection" against accidental shutdowns: a flick of the switch from power off will turn it on, but a flick from power on won't cut the power.

If the button press is not long enough for the voltage of C4 to drop significatively (below the Vthgs of the NMOS, datasheet doesn't specify but LTSpice says its ~2V for that current Bias point]), the circuit will remain latched.

[FlickWhileOn]


This is a very nice feature of this circuit, where a quick flick of the switch from unpowered will power the circuit on, but a quick press won't accidentally turn it off.

To power the circuit off, the switch will have to be held for a certain amount of time, so C4 charges enough (via R5-R6-R7-D1) to get V<sub>G-NMOS</sub> below ~2V. In the graph, V<sub>C4</sub> steadily rises and V<sub>G-NMOS</sub> decreases accordingly until it stops conducting. 

Without the NMOS to tie the PMOS' gate to GND, V<sub>A</sub> steadily rises until the PMOS cuts off. With that, V<sub>OUT</sub> collapses.

[TODO: show pspice plots for turning on and off]

Because we want to use this circuit to alert the Raspberry Pi, we need to lower the voltage to 3V3. Maybe a linear regulator is overkill, but hey! it gives us another chip to solder and they are cheap. The NMOS side of the circuit will be fed with 3V3, as its more than enough to trigger the MOSFET into conduction. The only drawback is that the RC circuit that it's holding the gate on has now less time to discharge to go below V<sub>TH-NMOS</sub>, as the time required to hold the switch to power the circuit off, from power on, is dictated by the discharge of C4 via R5-R6-R7 (there are three resistors ins series because I have 100kΩ resistors already laying around) and now it starts at 3V3 and not V<sub>IN</sub>. 

[TODO: code to calculate the C4/Rc4 values]

### Auto-On feature

Only the C2 cap, providing the Auto-On feature affects the gate of the PMOS at startup. If connected to ground, it will keep the gate to 0V and the PMOS will conduct when power is applied. If C2 is connected to V<sub>IN</sub> instead, then the gate wil be pulled to V<sub>IN</sub> volts, and turn on.

### Voltage limits in the Transistors

While the transistors are rated V<sub>DS</sub> = 40V, the max V<sub>GS</sub> is only 20V. The max V<sub>IN</sub> is 18V, so we are relatively safe (2V of margin!). We could connect a 15V zener to hold the voltage in place.

By powering the NMOS with the 3V3 rail, we won't require a diode on the NMOS.

### In-rush Protection

A description of the In-Rush Protection can be found [here](http://www.mosaic-industries.com/embedded-systems/microcontroller-projects/electronic-circuits/push-button-switch-turn-on/inrush-current-limited-mosfet).

### Power Dissipation in the MOSFETS

The NMOS is easy: it doesn't drive any loads, so it won't even notice it's being switched.

The PMOS is doing all the work here. The amp is a 2x25W, but I will never play it loud, it's too loud! Let's assume a max consumption of 3A (36W @12V). 

While the circuit is on, the low R<sub>DSON</sub> (0.1Ω for the PMOS) will make sure that it doesn't dissipate much power.

The critical moment is when the transistor goes through the "linear" region. During power on there won't be much power being drawn (whatever the Raspberry Pi requires, and it will be <100mA @12V). If we power of with a load of 3A, there will be a peak of power dissipated of 10W for 10-15ms!.

[TODO: graph]

Squaring the pulse to make integration easier, 10W for 15ms are $$10 W * .015 s = .150J$$. The datasheet lists a thermal resistance to ambient of $$90\frac{°C}/{W}$$, so this peak will rise the temperature of the transistor 14°C! Luckily it won't happen, and even if it happens it's not repeatable. 